# セグメント情報の読み取り手順のテスト

本ノートブックで、HTML化された決算短信からセグメント情報が読み取れるかテストします。

## 手順

1. 「報告セグメント」を含むHTMLテーブルを取得する
   * 処理: 「報告セグメント」の記載を内部に含むtableタグを取得する。
   * 解釈: 取得できない場合、会社独自様式で報告しているかセグメント情報の記載がない。
   * 事例: ディスコ(61460)は記載がそもそもなく、オービック(46840)はテーブルがない。
3. 「報告セグメント」のHTMLテーブルから、報告時期を取得する
   * 処理: 「報告セグメント」のtableタグ周辺の記載から報告時期を取得する。直前3行以内にある「当第」、「前第」で始まるテキストから読み取る。
   * 解釈: 取得できない場合、報告時期について明記されていないか想定された記載で書いていない。
4. 「報告セグメント」のHTMLテーブルから、セグメントの列を取得する
   * 処理: 「報告セグメント」のtableタグからセグメントの列を取得する。table内のcolspanでまとめられた列をセグメントとする。
   * 解釈: 取得できない場合、セグメントについて一般的ではない様式で記載していない。
5. 「報告セグメント」のHTMLテーブルから、売上・利益の行を取得する
   * 処理: 「報告セグメント」のtableタグから売上・利益について書かれた行を取得する。table内の最も左の列のうち、最初の空白ではない行タイトルを売り上げの勘定とし次の行を利益とする。ただし、「計」を含む場合は計を売上とし次行を利益とする。
   * 解釈: 取得できない場合、売上・利益について一般的ではない様式で記載していない。
6. 「報告セグメント」のHTMLテーブルから、各セグメントの売上・利益を取得する
   * 処理: 「報告セグメント」のtableタグからセグメントの列、売上・利益の行を指定し値を取得する。
   * 解釈: 4, 5が成功していれば該当のデータが取得できる。
7. Optional: セグメントについての記載を取得する
   * 処理: 4で取得したセグメント名を含む記載を取得する。セグメント名もしくは記号で囲まれた見出しから、次の空行もしくは次のセグメントについて記載がある箇所までをセグメントの説明として取得する。
   * 解釈: 取得できない場合、セグメントについて本文に記載がない。

## 検証対象

ランダムに選択した5社の`qualitative.htm`で検証する。

* sample1: セブンアンドアイ_2023年2月期 第1四半期決算短信〔日本基準〕（連結）
* sample2: ファーストリテイリング_2022年８月期　第３四半期決算短信〔IFRS〕（連結）
* sample3: オービック_2022年３月期 第3四半期決算短信[日本基準]（連結）
* sample4: ディスコ_2023年3月期第1四半期決算短信〔日本基準〕（連結）
* sample5: 野村総合研究所_2023年3月期 第1四半期決算短信〔ＩＦＲＳ〕（連結）

## 検証

### 1. 「報告セグメント」を含むHTMLテーブルを取得する

* 処理: 「報告セグメント」の記載を内部に含むtableタグを取得する。
* 解釈: 取得できない場合、会社独自様式で報告しているかセグメント情報の記載がない。
* 事例: ディスコ(61460)は記載がそもそもなく、オービック(46840)はテーブルがない。


まず、htmファイルを読み込む

In [1]:
from pathlib import Path
import unicodedata
import numpy as np
from bs4 import BeautifulSoup


def read_financial_result_html(path):
    _path = path if isinstance(path, Path) else Path(path)
    text = ""
    with _path.open() as r:
        lines = r.readlines()
        # 記載されたテキストの文字種を統一するため、Unicode正規化を行う
        lines = [unicodedata.normalize("NFKC",line) for line in lines]
        text = "".join(lines)
    
    html = BeautifulSoup(text)
    return html

In [2]:
htmls = []

target_path = Path("./data/raw")
for path in sorted(target_path.glob("*.htm")):
    htmls.append(read_financial_result_html(path))

次に、報告セグメントを含むテーブルを読み込む

In [3]:
def find_segment_tables(html):
    tables = html.find_all("table")
    segment_tables = []
    for t in tables:
        if t.find(text="報告セグメント"):
            segment_tables.append(t)
    
    return segment_tables

In [4]:
segment_tables_list = []

for html in htmls:
    segment_tables_list.append(find_segment_tables(html))

3番目のオービック、4番目のディスコには報告セグメントが存在しない。

In [5]:
np.testing.assert_array_equal([len(tables) > 0 for tables in segment_tables_list], [True, True, False, False, True])

AssertionError: 
Arrays are not equal

(shapes (0,), (5,) mismatch)
 x: array([], dtype=float64)
 y: array([ True,  True, False, False,  True])

取得できたもののみ次の処理対象とする。

In [ ]:
segment_tables_list = [tables for tables in segment_tables_list if len(tables) > 0]

報告セグメントで報告している場合、前期・当期の2期のテーブルのみ存在する。

In [ ]:
np.testing.assert_array_equal([len(tables) for tables in segment_tables_list], [2, 2, 2])

### 2. 「報告セグメント」のHTMLテーブルから、報告時期を取得する

* 処理: 「報告セグメント」のtableタグ周辺の記載から報告時期を取得する。直前3行以内にある「当第」、「前第」で始まるテキストから読み取る。
* 解釈: 取得できない場合、報告時期について明記されていないか想定された記載で書いていない。

In [ ]:
import re
from datetime import datetime


def read_table_period(table):
    FIND_LIMIT = 3
    PREVIOUS = "前第"
    CURRENT = "当第"
    FROM_PATTERN = re.compile("自.*?\d+年\d+月\d+日")
    TO_PATTERN = re.compile("至.*?\d+年\d+月\d+日")
    period = {
        "kind": "",
        "description": "",
        "begin_date": "",
        "end_date": ""
    }

    count = 0
    tag = table
    while count < FIND_LIMIT:
        p = tag.find_previous("p")
        text = p.string.strip()
        if re.match(f".*{PREVIOUS}.+", text):
            period["kind"] = "previous"
            period["description"] = text
        if re.match(f".*{CURRENT}.+", text):
            period["kind"] = "current"
            period["description"] = text
        
        if period["kind"]:
            if FROM_PATTERN.search(period["description"]):
                date_text = FROM_PATTERN.search(text).group(0).replace("自","").strip()
                period["begin_date"] = datetime.strptime(date_text, "%Y年%m月%d日")
            if TO_PATTERN.search(period["description"]):
                date_text = TO_PATTERN.search(text).group(0).replace("至","").strip()
                period["end_date"] = datetime.strptime(date_text, "%Y年%m月%d日")

            break
        count += 1
        tag = p
    
    return period

全てのHTMLについて前期/当期が存在する。

In [ ]:
for tables in segment_tables_list:
    for i, t in enumerate(tables):
        period = read_table_period(t)
        print(period)
        if i == 0:
            assert (period["kind"] == "previous")
        else:
            assert (period["kind"] == "current")

{'kind': 'previous', 'description': 'I 前第1四半期連結累計期間(自 2021年3月1日 至 2021年5月31日)', 'begin_date': datetime.datetime(2021, 3, 1, 0, 0), 'end_date': datetime.datetime(2021, 5, 31, 0, 0)}
{'kind': 'current', 'description': 'II 当第1四半期連結累計期間(自 2022年3月1日 至 2022年5月31日)', 'begin_date': datetime.datetime(2022, 3, 1, 0, 0), 'end_date': datetime.datetime(2022, 5, 31, 0, 0)}
{'kind': 'previous', 'description': '前第3四半期連結累計期間(自 2020年9月1日 至 2021年5月31日)', 'begin_date': datetime.datetime(2020, 9, 1, 0, 0), 'end_date': datetime.datetime(2021, 5, 31, 0, 0)}
{'kind': 'current', 'description': '当第3四半期連結累計期間(自 2021年9月1日 至 2022年5月31日)', 'begin_date': datetime.datetime(2021, 9, 1, 0, 0), 'end_date': datetime.datetime(2022, 5, 31, 0, 0)}
{'kind': 'previous', 'description': '前第1四半期連結累計期間(自 2021年4月1日 至 2021年6月30日)', 'begin_date': datetime.datetime(2021, 4, 1, 0, 0), 'end_date': datetime.datetime(2021, 6, 30, 0, 0)}
{'kind': 'current', 'description': '当第1四半期連結累計期間(自 2022年4月1日 至 2022年6月30日)', 'begin_date': datetime.da

以降は当期のテーブルを対象に検証する。

In [ ]:
current_tables = [tables[1] for tables in segment_tables_list]
assert len(current_tables) == 3

### 4.「報告セグメント」のHTMLテーブルから、セグメントの列を取得する

処理: 「報告セグメント」のtableタグからセグメントの列を取得する。table内のcolspanでまとめられた列をセグメントとする。
解釈: 取得できない場合、セグメントについて一般的ではない様式で記載していない。

In [ ]:
def read_table_segments(table):
    merged_cell = table.find_all("td", colspan=True)
    SEGMENT_TEXT = "報告セグメント"
    EXCLUDES = re.compile(".{0,2}計$")
    NORMALIZER = re.compile("\s|\r|\n")
    segments = []
    
    segment_cells = [cell for cell in merged_cell if SEGMENT_TEXT in cell.text]
    if len(segment_cells) == 0:
        return segments
    else:
        cell = segment_cells[0]
        num_segments = int(cell.attrs["colspan"])
        descriptions = [d.text.strip() for d in cell.find_previous("tr").find_all("td")]
        segment_start = descriptions.index(SEGMENT_TEXT)
        title_row = cell.find_next("tr")
        segment_index = 0
        for i, title in enumerate(title_row.find_all("td")):
            if i < segment_start:
                continue
            elif i < (i + num_segments):
                segment_name = NORMALIZER.sub("", title.text.strip())
                if segment_name and not EXCLUDES.match(segment_name):
                    segments.append({
                        "segment_index": segment_index,
                        "column": i,
                        "segment_name": segment_name
                    })
                    segment_index += 1
    
    return segments

In [ ]:
np.testing.assert_array_equal(
    [s["segment_name"] for s in read_table_segments(current_tables[0])],
    ["国内コンビニエンスストア事業", "海外コンビニエンスストア事業", "スーパーストア事業",
     "百貨店・専門店事業", "金融関連事業", "その他の事業"]
)
np.testing.assert_array_equal(
    [s["segment_name"] for s in read_table_segments(current_tables[1])],
    ["国内ユニクロ事業", "海外ユニクロ事業", "ジーユー事業", "グローバルブランド事業"]
)
np.testing.assert_array_equal(
    [s["segment_name"] for s in read_table_segments(current_tables[2])],
    ["コンサルティング", "金融ITソリューション", "産業ITソリューション", "IT基盤サービス"]
)

### 5. 「報告セグメント」のHTMLテーブルから、売上・利益の行を取得する
   * 処理: 「報告セグメント」のtableタグから売上・利益について書かれた行を取得する。table内の最も左の列のうち、最初の空白ではない行タイトルを売り上げの勘定とし次の行を利益とする。ただし、「計」を含む場合は計を売上とし次行を利益とする。
   * 解釈: 取得できない場合、売上・利益について一般的ではない様式で記載していない。


In [ ]:
def read_table_sales_profit(table):
    SUM = re.compile(".{0,2}計$")
    NORMALIZER = re.compile("\s|\r|\n")
    UNIT_TEXT = "単位"
    unit = 1000000

    merged_cell = table.find_all("td", colspan=True)
    unit_cells = [cell for cell in merged_cell if UNIT_TEXT in cell.text]
    
    if len(unit_cells) > 0:
        cell = unit_cells[0]
        if "千円" in cell.text:
            unit = 1000
        elif "十億" in cell.text:
            unit = 1000000000
    
    rows = table.find_all("tr")
    row_headers = [row.find_next("td") for row in rows]
    header_texts = [NORMALIZER.sub("", cell.text) for cell in row_headers]
    sales_index = 0
    sales_name = ""
    for i, text in enumerate(header_texts):
        if text and not sales_name:
            sales_name = text
            sales_index = i
        elif SUM.match(text):
            sales_index = i
            break
    
    values = []
    values.append({
        "value_index": 0,
        "row": sales_index,
        "value_kind": "Sales",
        "value_name": sales_name,
        "unit": unit
    })
    values.append({
        "value_index": 1,
        "row": sales_index + 1,
        "value_kind": "Profit",
        "value_name": header_texts[sales_index + 1],
        "unit": unit
    })
    
    return values

In [ ]:
np.testing.assert_array_equal(
    [s["value_name"] for s in read_table_sales_profit(current_tables[0])],
    ["営業収益", "セグメント利益又は損失(△)"]
)
np.testing.assert_array_equal(
    [s["value_name"] for s in read_table_sales_profit(current_tables[1])],
    ["売上収益", "営業利益又は損失(△)"]
)
np.testing.assert_array_equal(
    [s["value_name"] for s in read_table_sales_profit(current_tables[2])],
    ["売上収益", "営業利益"]
)

### 6. 「報告セグメント」のHTMLテーブルから、各セグメントの売上・利益を取得する

* 処理: 「報告セグメント」のtableタグからセグメントの列、売上・利益の行を指定し値を取得する。
* 解釈: 4, 5が成功していれば該当のデータが取得できる。


In [ ]:
import pandas as pd


def read_segment_sales_profit(table):
    segment_data = []
    segments = read_table_segments(table)
    accounts = read_table_sales_profit(table)
    for s in segments:
        for a in accounts:
            cell = table.find_all("tr")[a["row"]].find_all("td")[s["column"]]
            result = {}
            result.update(s)
            result.update(a)
            value = cell.text.strip().replace("-", "").replace(",", "").replace("△", "-")
            try:
                result["value"] = float(value)
            except Exception as ex:
                result["value"] = None
            segment_data.append(result)

    return pd.DataFrame(segment_data)

In [ ]:
read_segment_sales_profit(current_tables[0])

,segment_index,column,segment_name,value_index,row,value_kind,value_name,unit,value
0,0,1,国内コンビニエンスストア事業,0,6,Sales,営業収益,1000000,215243.0
1,0,1,国内コンビニエンスストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,59282.0
2,1,2,海外コンビニエンスストア事業,0,6,Sales,営業収益,1000000,1723889.0
3,1,2,海外コンビニエンスストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,43981.0
4,2,3,スーパーストア事業,0,6,Sales,営業収益,1000000,355772.0
5,2,3,スーパーストア事業,1,7,Profit,セグメント利益又は損失(△),1000000,3517.0
6,3,4,百貨店・専門店事業,0,6,Sales,営業収益,1000000,112904.0
7,3,4,百貨店・専門店事業,1,7,Profit,セグメント利益又は損失(△),1000000,1086.0
8,4,5,金融関連事業,0,6,Sales,営業収益,1000000,47560.0
9,4,5,金融関連事業,1,7,Profit,セグメント利益又は損失(△),1000000,9205.0


In [ ]:
read_segment_sales_profit(current_tables[1])

,segment_index,column,segment_name,value_index,row,value_kind,value_name,unit,value
0,0,1,国内ユニクロ事業,0,3,Sales,売上収益,1000000,640972.0
1,0,1,国内ユニクロ事業,1,4,Profit,営業利益又は損失(△),1000000,119067.0
2,1,2,海外ユニクロ事業,0,3,Sales,売上収益,1000000,841274.0
3,1,2,海外ユニクロ事業,1,4,Profit,営業利益又は損失(△),1000000,132793.0
4,2,3,ジーユー事業,0,3,Sales,売上収益,1000000,190545.0
5,2,3,ジーユー事業,1,4,Profit,営業利益又は損失(△),1000000,17852.0
6,3,4,グローバルブランド事業,0,3,Sales,売上収益,1000000,90084.0
7,3,4,グローバルブランド事業,1,4,Profit,営業利益又は損失(△),1000000,720.0


In [ ]:
read_segment_sales_profit(current_tables[2])

,segment_index,column,segment_name,value_index,row,value_kind,value_name,unit,value
0,0,1,コンサルティング,0,6,Sales,売上収益,1000000,9711.0
1,0,1,コンサルティング,1,7,Profit,営業利益,1000000,1859.0
2,1,3,金融ITソリューション,0,6,Sales,売上収益,1000000,81490.0
3,1,3,金融ITソリューション,1,7,Profit,営業利益,1000000,11678.0
4,2,5,産業ITソリューション,0,6,Sales,売上収益,1000000,68841.0
5,2,5,産業ITソリューション,1,7,Profit,営業利益,1000000,6679.0
6,3,7,IT基盤サービス,0,6,Sales,売上収益,1000000,40403.0
7,3,7,IT基盤サービス,1,7,Profit,営業利益,1000000,5652.0
